<a href="https://colab.research.google.com/github/danielbehargithub/LinkedIn_Salary/blob/main/Profile_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playwright
!playwright install


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 13.1 MB/s eta 0:00:00
161.3 MiB [] 0% 10.6s161.3 MiB [] 0% 41.5s161.3 MiB [] 0% 24.8s161.3 MiB [] 0% 15.7s161.3 MiB [] 0% 12.0s161.3 MiB [] 1% 8.1s161.3 MiB [] 1% 7.0s161.3 MiB [] 1% 6.3s161.3 MiB [] 2% 5.5s161.3 MiB [] 2% 5.1s161.3 MiB [] 3% 4.8s161.3 MiB [] 3% 4.5s161.3 MiB [] 4% 4.2s161.3 MiB [] 5% 4.0s161.3 MiB [] 5% 3.9s161.3 MiB [] 6% 3.8s161.3 MiB [] 6% 4.2s161.3 MiB [] 7% 4.2s161.3 MiB [] 7% 4.3s161.3 MiB [] 7% 4.2s161.3 MiB [] 8% 4.1s161.3 MiB [] 8% 4.0s161.3 MiB [] 9% 3.9s161.3 MiB [] 9% 4.0s161.3 MiB [] 10% 4.0s161.3 MiB [] 10% 3.9s161.3 MiB [] 10% 4.0s161.3 MiB [] 11% 3.9s161.3 MiB [] 12% 3.8s161.3 MiB [] 12% 3.7s161.3 MiB [] 13% 3.6s161.3 MiB [] 14% 3.4s161.3 MiB [] 14% 3.6s161.3 MiB [] 15% 3.5s161.3 MiB [] 15% 3.4s161.3 MiB [] 16% 3.3s161.3 MiB [] 17% 3.2s161.3 MiB [] 17% 3.1s161.3 MiB [] 18% 3.1s161.3 MiB [] 19% 3.0s161.3 MiB [] 20% 2.9s161.3 MiB [] 21% 2.9s161.3 MiB [] 21% 2.8s161.3 MiB [] 22% 2.8s161.3 MiB [] 23%

In [ ]:
# Enter url, email and password
user_url = "https://www.linkedin.com/in/daniel-behar-168647280"
email = "daniel10behar@gmail.com"
password = "..."


In [ ]:
import nest_asyncio
from playwright.async_api import async_playwright
import asyncio

async def scrape_user_profile(user_url, email, password):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # Running in headless mode
        page = await browser.new_page()

        print("Navigating to LinkedIn login page...")
        # Navigate to LinkedIn login page
        await page.goto("https://www.linkedin.com/login")
        await asyncio.sleep(3)

        # Perform login
        print("Logging into LinkedIn...")
        await page.fill('input[id="username"]', email)
        await page.fill('input[id="password"]', password)
        await page.click('button[type="submit"]')
        await asyncio.sleep(5)

        # Check for two-step verification
        if await page.is_visible('input[name="pin"]'):
            print("Two-step verification detected. Please enter the code sent to your email.")
            verification_code = input("Enter the verification code: ")
            await page.fill('input[name="pin"]', verification_code)
            await page.click('button[type="submit"]')
            await asyncio.sleep(3)

        # Relevant sections to scrape for the user
        profile_sections = [
            "/details/education/",
            "/details/skills/",
            "/details/experience/",
            # "/details/certifications/"
        ]

        scraped_data = {}  # Dictionary to store HTML content of each section

        # מספר ניסיונות להורדת HTML
        MAX_RETRIES = 3

        # Loop through the list of profile URLs
        for section in profile_sections:
            url = f"{user_url}{section}"
            retries = 0

            while retries < MAX_RETRIES:
                try:
                    print(f"Navigating to section: {url} (Attempt {retries + 1})")

                    # Navigate to the section URL
                    await page.goto(url)
                    await asyncio.sleep(3)

                    # Get page title to verify if the correct page is loaded
                    page_title = await page.title()
                    section_name = section.strip("/").split("/")[-1]  # Extract section name
                    print(f"Page title: {page_title}")
                    print(f"Section name: {section_name}")

                    # Verify if the page is correct
                    if section_name.capitalize() not in page_title:
                        print(f"Page {url} did not load correctly. Retrying...")
                        retries += 1
                        await asyncio.sleep(5)  # Wait a bit before retrying
                        continue  # Retry the same section

                    # Save the HTML content of the section page
                    html = await page.content()
                    scraped_data[section_name] = html  # Store HTML content in dictionary
                    print(f"HTML content for {section_name} saved in memory.")

                    # Save the HTML file for debugging
                    filename = f"{section_name}.html"
                    with open(filename, "w", encoding="utf-8") as f:
                        f.write(html)
                    print(f"Saved HTML for {url} as {section_name}")

                    # Take a screenshot of the section page
                    screenshot_filename = f"{section_name}.png"
                    await page.screenshot(path=screenshot_filename, full_page=True)
                    print(f"Saved screenshot for {url} as {screenshot_filename}")

                    break  # If successful, exit the retry loop

                except Exception as e:
                    print(f"Failed to scrape {url}: {e}")
                    retries += 1
                    await asyncio.sleep(5)  # Wait a bit before retrying

            if retries == MAX_RETRIES:
                print(f"Failed to scrape {url} after {MAX_RETRIES} attempts. Skipping...")

        # Close the browser
        print("Closing the browser...")
        await browser.close()

        return scraped_data  # Return all scraped HTML as a dictionary



In [ ]:
scraped_html_data = await scrape_user_profile(user_url, email, password)

from bs4 import BeautifulSoup

# Example HTML from the dictionary
html_skills = scraped_html_data.get("skills")
html_education = scraped_html_data.get("education")
html_experience = scraped_html_data.get("experience")

# Parse HTML
soup_skill = BeautifulSoup(html_skills, "html.parser")
soup_education = BeautifulSoup(html_education, "html.parser")
soup_experience = BeautifulSoup(html_experience, "html.parser")


Navigating to LinkedIn login page...
Logging into LinkedIn...
Navigating to section: https://www.linkedin.com/in/daniel-behar-168647280/details/education/ (Attempt 1)
Page title: (13) Education | Daniel Behar | LinkedIn
Section name: education
HTML content for education saved in memory.
Saved HTML for https://www.linkedin.com/in/daniel-behar-168647280/details/education/ as education
Saved screenshot for https://www.linkedin.com/in/daniel-behar-168647280/details/education/ as education.png
Navigating to section: https://www.linkedin.com/in/daniel-behar-168647280/details/skills/ (Attempt 1)
Page title: (13) Skills | Daniel Behar | LinkedIn
Section name: skills
HTML content for skills saved in memory.
Saved HTML for https://www.linkedin.com/in/daniel-behar-168647280/details/skills/ as skills
Saved screenshot for https://www.linkedin.com/in/daniel-behar-168647280/details/skills/ as skills.png
Navigating to section: https://www.linkedin.com/in/daniel-behar-168647280/details/experience/ (Att

In [ ]:
experience_section = soup_experience.find("main", {"aria-label": "Experience"})

company_records = experience_section.find_all("li", class_="pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column")

for company in company_records:
    # שם החברה
    company_name_container = company.find("div", class_="display-flex align-items-center mr1 hoverable-link-text t-bold") or \
                         company.find("div", class_="display-flex align-items-center mr1 t-bold")
    company_name = (
        company_name_container.find("span", {"aria-hidden": "true"}).get_text(strip=True)
        if company_name_container else "N/A"
    )


    # משך העבודה וסוג העבודה
    duration_container = company.find("span", class_="t-14 t-normal")
    if duration_container:
        duration_text = duration_container.find("span", {"aria-hidden": "true"}).get_text(strip=True)
        if "·" in duration_text:
            job_type, job_duration = [part.strip() for part in duration_text.split("·", 1)]
        else:
            job_type, job_duration = "N/A", duration_text
    else:
        job_type, job_duration = "N/A", "N/A"


    additional_content_container = company.find("div", class_="display-flex align-items-center t-14 t-normal t-black")
    additional_content = (
        additional_content_container.get_text(strip=True) if additional_content_container else "N/A"
    )


    # הצגת התוצאות
    print(f"Company Name: {company_name}")
    print(f"Job Duration: {job_duration}")
    print(f"Job Type: {job_type}")
    print(f"Additional Content: {additional_content}")
    print("-" * 50)


Company Name: Israel Defense Forces
Job Duration: 4 yrs
Job Type: N/A
Additional Content: N/A
--------------------------------------------------


In [ ]:
education_section = soup_education.find("main", {"aria-label": "Education"})

education_records = education_section.find_all("div", {"data-view-name": "profile-component-entity"})

for record in education_records:
    print("Education record:")

    # Institution name
    institution_container = record.find("div", class_="display-flex align-items-center mr1 hoverable-link-text t-bold")
    institution_name = institution_container.get_text(strip=True) if institution_container else "N/A"
    print(f"  Institution: {institution_name}")

    # Date range
    date_container = record.find("span", class_="pvs-entity__caption-wrapper")
    date_range = date_container.get_text(strip=True) if date_container else "N/A"
    print(f"  Date: {date_range}")

    # Additional description
    description_container = record.find("span", class_="t-14 t-normal")
    description_text = description_container.get_text(strip=True) if description_container else "N/A"
    print(f"  Description: {description_text}")

    # Skills
    skills_container = record.find("div", class_="display-flex align-items-center t-14 t-normal t-black")
    skills = []
    if skills_container:
        skills_span = skills_container.find("span", {"aria-hidden": "true"})
        if skills_span:
            skills_text = skills_span.get_text(strip=True)
            if "Skills:" in skills_text:
                skills_text = skills_text.replace("Skills:", "").strip()
            skills = [skill.strip() for skill in skills_text.split("·")]
    print(f"  Skills: {', '.join(skills)}")

    # Additional text
    additional_text_container = record.find("div", class_="inline-show-more-text--is-collapsed")
    additional_text = ""
    if additional_text_container:
        additional_span = additional_text_container.find("span", {"aria-hidden": "true"})
        if additional_span:
            additional_text = additional_span.get_text(" ", strip=True)
    print(f"  Additional Text: {additional_text}")


Education record:
  Institution: Technion - Israel Institute of TechnologyTechnion - Israel Institute of Technology
  Date: Oct 2020
  Description: N/A
  Skills: Statistical Data Analysis, Apache Spark, Java, Data Structures, PyTorch, Deep Learning, Django
  Additional Text: 


In [ ]:
# Find all <a> tags with data-field="skill_page_skill_topic"
skill_links = soup_skill.find_all("a", {"data-field": "skill_page_skill_topic"})

# Extract skills
skills = []
for link in skill_links:
    # Find the <span> with aria-hidden="true" inside the link
    skill_span = link.find("span", {"aria-hidden": "true"})
    if skill_span:
        skill_text = skill_span.get_text(strip=True)
        skills.append(skill_text)

unique_skills = sorted(set(skills))
print("Unique skills:", unique_skills)


Unique skills: ['Apache Spark', 'Data Analysis', 'Data Structures', 'Deep Learning', 'Django', 'HTML', 'Java', 'PyTorch', 'Python (Programming Language)', 'SQL', 'Statistical Data Analysis']
